# <center><b> <em> Exploratory Data Analysis </em> </b></center>

### <font color="green"> | - Import des Biblio et Dataframe</font>

In [15]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [16]:
# Read the XLSX file
data = pd.read_csv("../data/silver.csv")

In [17]:
data.shape

(797885, 8)

In [21]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 797885 entries, 0 to 797884
Data columns (total 8 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   invoice       797885 non-null  object 
 1   stock_code    797885 non-null  object 
 2   description   797885 non-null  object 
 3   quantity      797885 non-null  int64  
 4   invoice_date  797885 non-null  object 
 5   unit_price    797885 non-null  float64
 6   customer_id   797885 non-null  float64
 7   country       797885 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 48.7+ MB


### <font color="green"> || - Categorical Variable Analysis : </font>

In [18]:
data.describe()

,quantity,unit_price,customer_id
count,797885.000000,797885.000000,797885.000000
mean,12.602980,3.702732,15313.062777
std,191.670371,71.392549,1696.466663
min,-80995.000000,0.000000,12346.000000
25%,2.000000,1.250000,13964.000000
50%,5.000000,1.950000,15228.000000
75%,12.000000,3.750000,16788.000000
max,80995.000000,38970.000000,18287.000000


### <font color="green"> ||| - Numerical Variable Analysis : </font>

In [19]:
selected_data = data[data["invoice"].str.contains("C", na=False)]

In [20]:
df = data[~data["invoice"].str.contains("C", na = False)]

##### <font color="yellow">l’hypothèse nulle est : « les deux variables testées sont indépendantes » </font>